<a href="https://colab.research.google.com/github/rajeswarimuppidi/Python-Assessment/blob/main/Currency_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# # Currency Converter

import pandas as pd
import re
from tkinter import *
import tkinter as tk
from tkinter import ttk

class RealTimeCurrencyConverter():
    '''
    This is class is for converting currencies.
    '''
    
    def __init__(self,conversion_rate,currencies,cross_currency_df):
        self.conversion_rate = conversion_rate
        self.currencies = currencies
        self.cross_currency_df = cross_currency_df

    def convert(self, from_currency, to_currency, amount): 
      # limiting the precision to 2 decimal places 
      
      conversion_amount = ''
      if (from_currency + to_currency) in self.conversion_rate.keys():
        conversion_amount = round((amount * self.conversion_rate[(from_currency + to_currency)]), 2) 
        print('in if :',conversion_amount)
        
      elif (to_currency + from_currency) in self.conversion_rate.keys():
          conversion_amount = round(((1/ self.conversion_rate[(to_currency + from_currency)]) * amount), 2) 
          print('in else if:',conversion_amount)

      else :
        for i, rows in self.cross_currency_df.iterrows():
            if rows.Name == from_currency:
                cross_currency_cty = rows[to_currency]
                if cross_currency_cty == '1:1':
                    conversion_amount = round(amount,2)
                elif cross_currency_cty != 'Inv' or cross_currency_cty != 'D':
                    if (from_currency + cross_currency_cty) in conversion_rate.keys():
                        conversion_amount_tmp = round((amount * self.conversion_rate[(from_currency + cross_currency_cty)]), 2) 
                        print('conversion_amount_tmp:',conversion_amount_tmp)
                        if (cross_currency_cty + to_currency) in conversion_rate.keys():
                            conversion_amount = round((conversion_amount_tmp *  self.conversion_rate[(cross_currency_cty + to_currency)]), 2) 
                            print('conversion_amount:',conversion_amount)
                        elif (to_currency + cross_currency_cty) in conversion_rate.keys():
                            conversion_amount = round(((1/ self.conversion_rate[(to_currency + cross_currency_cty)]) * conversion_amount_tmp), 2) 
                    elif(cross_currency_cty + from_currency) in conversion_rate.keys():
                        conversion_amount_tmp = round(((1/ self.conversion_rate[(cross_currency_cty + from_currency)]) * amount) , 2) 
                        if (cross_currency_cty + to_currency) in conversion_rate.keys():
                            conversion_amount = round((conversion_amount_tmp  * self.conversion_rate[(cross_currency_cty + to_currency)]), 2) 
                        elif (to_currency + cross_currency_cty) in conversion_rate.keys():
                            conversion_amount = round(((1/ self.conversion_rate[(to_currency + cross_currency_cty)])  * conversion_amount_tmp), 2)
                
                elif cross_currency_cty == 'Inv':
                    conversion_amount = round(((1/ self.conversion_rate[(to_currency + from_currency)]) * amount), 2) 
                    print('in else if:',conversion_amount)
                elif cross_currency_cty == 'D':
                    conversion_amount = round((amount * self.conversion_rate[(from_currency + to_currency)]), 2) 
            
      if conversion_amount is not None:
          return conversion_amount  
      else:
          print('no conversion amount')
    
        

class App(tk.Tk):
    '''
    This class is for UI.
    '''
	
    def __init__(self, converter):
        tk.Tk.__init__(self)
        self.title = 'Currency Converter'
        self.currency_converter = converter

        #self.configure(background = 'blue')
        self.geometry("500x200")
        
        # Label
        self.intro_label = Label(self, text = 'Python FX Calculator',  fg = 'blue', relief = tk.GROOVE, borderwidth = 5,justify = tk.CENTER)
        self.intro_label.config(font = ('Courier',15,'bold'))

       # self.date_label = Label(self, text = f"Date : {self.currency_converter.data['date']}", relief = tk.GROOVE, borderwidth = 5)

        self.intro_label.place(x = 140 , y = 5)
        #self.date_label.place(x = 160, y= 50)

        # Entry box
        valid = (self.register(self.restrictNumberOnly), '%d', '%P')
        self.amount_field = Entry(self,bd = 3, relief = tk.RIDGE, justify = tk.CENTER,validate='key', validatecommand=valid)
        self.converted_amount_field_label = Label(self, text = '', fg = 'black', bg = 'white', relief = tk.RIDGE, justify = tk.CENTER, width = 17, borderwidth = 3)

        # dropdown
        self.from_currency_variable = StringVar(self)
        self.from_currency_variable.set("AUD") # default value
        self.to_currency_variable = StringVar(self)
        self.to_currency_variable.set("USD") # default value

        font = ("Courier", 12, "bold")
        self.option_add('*TCombobox*Listbox.font', font)
        self.from_currency_dropdown = ttk.Combobox(self, textvariable=self.from_currency_variable,values=currencies, font = font, state = 'readonly', width = 12, justify = tk.CENTER)
        self.to_currency_dropdown = ttk.Combobox(self, textvariable=self.to_currency_variable,values=currencies, font = font, state = 'readonly', width = 12, justify = tk.CENTER)

        # placing
        self.from_currency_dropdown.place(x = 30, y= 120)
        self.amount_field.place(x = 36, y = 150)
        self.to_currency_dropdown.place(x = 340, y= 120)
        #self.converted_amount_field.place(x = 346, y = 150)
        self.converted_amount_field_label.place(x = 346, y = 150)
        
        # Convert button
        self.convert_button = Button(self, text = "Convert", fg = "black", command = self.perform) 
        self.convert_button.config(font=('Courier', 10, 'bold'))
        self.convert_button.place(x = 225, y = 135)

    def perform(self):
        amount = float(self.amount_field.get())
        from_curr = self.from_currency_variable.get()
        to_curr = self.to_currency_variable.get()

        converted_amount = self.currency_converter.convert(from_curr,to_curr,amount)
        converted_amount = round(converted_amount, 2)

        self.converted_amount_field_label.config(text = str(converted_amount))
    
    def restrictNumberOnly(self, action, string):
        regex = re.compile(r"[0-9,]*?(\.)?[0-9,]*$")
        result = regex.match(string)
        return (string == "" or (string.count('.') <= 1 and result is not None))

if __name__ == '__main__':
    conversion_rate = { "AUDUSD" : 0.8371, 
                   "CADUSD":0.8711,
                   "USDCNY":6.1715 ,
                   "EURUSD":1.2315 ,
                   "GBPUSD":1.5683 ,
                   "NZDUSD":0.7750 ,
                   "USDJPY":119.95 ,
                   "EURCZK":27.6028 ,
                   "EURDKK":7.4405 ,
                  "EURNOK":8.6651
	}
    currencies = ['AUD','CAD','CNY','CZK','DKK','EUR', 'GBP', 'JPY', 'NOK', 'NZD', 'USD']
    cross_currency_df = pd.read_csv('cross_currency_data.csv')
    converter = RealTimeCurrencyConverter(conversion_rate,currencies,cross_currency_df)

    App(converter)
    mainloop()



TclError: ignored